## Дерево стандартных исключений

Как нетрудно было догадаться, исключения представлены определёнными классами, которые в той или иной степени наследуются от `BaseException`.

Классы `+-- SystemExit +-- KeyboardInterrupt +-- GeneratorExit` — являются исключениями, которые нельзя поймать, т.к. их возникновение не зависит от выполнения программы. А все, что наследуются от `Exception`, можно отловить и обработать (хорошенько так). Другой вопрос, что некоторые из них возникают очень редко.

Здесь главное понять, что ловить в блоке `except` можно не только сам класс, но и его родителя, например:

In [2]:
try:
    raise ZeroDivisionError # возбуждаем исключение ZeroDivisionError
except ArithmeticError: # ловим его родителя
    print("Hello from arithmetic error")

Hello from arithmetic error


Результат:

Такой способ отлова будет работать прекрасно. Другое дело, что делать так лучше не стоит, потому что вы рискуете упустить детали. Действуя от обратного, однако, надо быть осторожным. Если, например, надо поймать несколько эксепшенов, то идти следует **вверх** по дереву.

Например:

In [3]:
try:
    raise ZeroDivisionError
except ZeroDivisionError: # сначала пытаемся поймать наследника
    print("Zero division error")
except ArithmeticError: # потом ловим потомка
    print("Arithmetic error")

Zero division error


Это всё, что хотелось ещё рассказать о конструкции `try-except`.

Если кратко обобщить, то можно сказать так: исключения — это тоже классы. Только поведение их особое: они могут наследоваться; ловить можно как сам класс, так и его родителя (при том в любом колене). В этом случае надо чтобы сначала обрабатывались более конкретные исключения, иначе они могут быть перекрыты их родителями и просто-напросто будут упущены.

Иногда может понадобиться написать собственное исключение. Например, вы пишите собственную игру и вам нужно обработать ситуацию, когда ваш персонаж пытается выучить способность, при этом не достигнув нужного уровня. Или пытается положить в уже заполненный инвентарь ещё какой-то предмет.

Принцип написания и отлова собственного исключения следующий:

In [4]:
class MyException(Exception): # создаём пустой класс – исключения 
    pass
 
try:
    raise MyException("message") # поднимаем наше исключение
except MyException as e: # ловим его за хвост как шкодливого котёнка
    print(e) # выводим информацию об исключении

message


Лучше всего, чтобы исключения были связаны между собой, т.е. наследоваться от общего класса исключения. Если продолжить предыдущий пример, то общим классом был бы `GameplayException` (вспоминаем пример с игрой из прошлого абзаца).

Наследуются исключения для того, чтобы можно было, продолжая всё тот же пример, отлавливать отдельно игровые исключения и отдельно исключения, касающееся ресурсов (закончилась оперативная память, место на диске и т.д.).

Давайте теперь попробуем построить собственные исключения с наследованием:

In [5]:
class ParentException(Exception): # создаём пустой класс – исключения потомка, наследуемся от exception
    pass
 
class ChildException(ParentException): # создаём пустой класс – исключение наследника, наследуемся от ParentException
    pass
 
try:
    raise ChildException("message") # поднимаем исключение-наследник
except ParentException as e: # ловим его родителя
    print(e) # выводим информацию об исключении

message


В этом случае мы успешно обработали собственный класс-наследник, хотя он и не является `ParentException`. Просто когда исключение возникает, в каждом блоке `except` по порядку интерпретатор проверяет, является ли исключение наследником или самим классом отлавливаемого исключения, и если да, то выполняет код в `except`.

Кстати говоря, класс с самописным исключением не обязательно должен быть пустым. Если вы хотите добавить собственные аргументы в конструктор, дополнительно произвести какие либо операции, то можете спокойно это делать, главное не забыть о нескольких нюансах:

In [9]:
class ParentException(Exception):
    def __init__(self, message, error): # допишем к нашему пустому классу конструктор, который будет печатать дополнительно в консоль информацию об ошибке.
        super().__init__(message) # помним про вызов конструктора родительского класса
        print(f"Errors: {error}") # печатаем ошибку
 
class ChildException(ParentException): # создаём пустой класс – исключение наследника, наследуемся от ParentException
    def __init__(self, message, error):
        super().__init__(message, error)
        self.message = message
 
try:
    raise ChildException("message", "error") # поднимаем исключение-наследник, передаём дополнительный аргумент
except ParentException as e:
    print(e) # выводим информацию об исключении

Errors: error
message
